In [ ]:
import os
import pandas as pd

In [ ]:
repository_path = '../repositories'
for root, dirs, files in os.walk(repository_path):
    for file in files:
        if '01.summarize.ipynb' in file:
            file_path = os.path.join(root, file)
            print(f'Processing {file_path}')
            os.system(
                f'cd {os.path.dirname(file_path)} && jupyter nbconvert --to notebook --execute --inplace {file}'
            )

Processing ../repositories/FITS/notebooks/01.summarize.ipynb


[NbConvertApp] Converting notebook 01.summarize.ipynb to notebook


In [ ]:

if not os.path.exists(repository_path):
    raise FileNotFoundError(f'This is not parent directory of {repository_path}. Please run this script in the parent directory of {repository_path}.')

metrics = {}
for root, dirs, files in os.walk(repository_path):
    for file in files:
        if file == '01.metrics.xlsx' and 'notebooks' in root:
            file_path = os.path.join(root, file)
            model = os.path.basename(os.path.dirname(root))
            model_metrics = {} # {sheet_name: {metric_name: metric_value}}
            df = pd.read_excel(file_path, sheet_name=None)
            for sheet_name, data in df.items():
                if sheet_name not in model_metrics:
                    model_metrics[sheet_name] = {}
                for col in data.columns:
                    if col not in model_metrics[sheet_name]:
                        model_metrics[sheet_name][col] = []
                    model_metrics[sheet_name][col].extend(data[col].tolist())
            for sheet_name, metrics_data in model_metrics.items():
                if sheet_name not in metrics:
                    metrics[sheet_name] = {}
                for metric_name, metric_values in metrics_data.items():
                    if metric_name not in metrics[sheet_name]:
                        metrics[sheet_name][metric_name] = []
                    if 'str' in metric_name:
                        metric_values = [str(v) for v in metric_values]
                    metrics[sheet_name][metric_name].extend(metric_values)
                    

In [ ]:
with pd.ExcelWriter('02.metrics.xlsx') as writer:
    for sheet_name, metrics_data in metrics.items():
        df = pd.DataFrame.from_dict(metrics_data, orient='index')
        df = df.transpose()
        df = df.reset_index()
        df = df.sort_values(by=['pred_len', 'mse-str', 'seq_len'], ascending=[True, True, True])
        
        df.to_excel(writer, sheet_name=sheet_name, index=False)